In [30]:
import pandas as pd
import json

lucene_lib = "lucene-9.8.0"
tantivy_lib = "tantivy-0.21"
metric_label_microsec="(μs)"
lucene_metric_label = f"{lucene_lib}{metric_label_microsec}"
tantivy_metric_label = f"{tantivy_lib}{metric_label_microsec}"

results_location = "../web/build/results.json"

In [32]:
#compare mean between tantivy and lucene durations
def get_mean_per_query_compare(test_type, library):
    results=json.load(open(results_location, "r"))
    df = results[test_type][library]
    df = pd.json_normalize(df)
    df = df.drop("tags", axis=1)
    df1 = pd.concat([pd.DataFrame(df["duration"].values.tolist())], 
             axis=1, 
             keys=df.columns)
    df1.columns = [f'{i}{j}' for i, j in df1.columns]
    metric_label = f"{library}{metric_label_microsec}"
    df[metric_label] = df1.mean(axis=1)
    df = df.drop(["duration","count"], axis=1)
    return df, metric_label

In [33]:
lucene_df, lucene_metric_label = get_mean_per_query_compare("TOP_100", lucene_lib)
tantivy_df, tantivy_metric_label = get_mean_per_query_compare("TOP_100", tantivy_lib)
df = lucene_df.merge(tantivy_df, on="query")
df

,query,lucene-9.8.0(μs),tantivy-0.21(μs)
0,ref,1021.0,324.1
1,http,120.5,72.8
2,from,4283.0,1343.8
3,title,2693.3,652.2
4,name,2271.8,625.3
...,...,...,...
6160,"""editor first""~4",25887.1,13108.6
6161,"""same name""",42404.2,34338.3
6162,"""same name""~4",49223.8,34664.0
6163,"""larger than""",13265.7,13003.9


In [34]:
df.describe()

,lucene-9.8.0(μs),tantivy-0.21(μs)
count,6165.000000,6165.000000
mean,13384.810251,9395.468500
std,25734.367381,23432.986015
min,12.400000,5.100000
25%,1209.500000,411.700000
50%,3415.300000,1390.300000
75%,15169.200000,8401.200000
max,364744.800000,318468.500000


In [35]:
metrics = pd.DataFrame(df, columns=[lucene_metric_label, tantivy_metric_label])
metrics.quantile([1, 0.99, 0.9, 0.75, 0.5])

,lucene-9.8.0(μs),tantivy-0.21(μs)
1.00,364744.800,318468.500
0.99,137322.664,135847.344
0.90,35325.920,23253.120
0.75,15169.200,8401.200
0.50,3415.300,1390.300


In [46]:
import plotly.express as px

px.scatter(df, y=[lucene_metric_label, tantivy_metric_label], hover_name="query",
          labels={
                  "value": f"latency {metric_label_microsec}",
                  "variable": "library",
                  "index": "dataframe query ID"
          })

